In [1]:
from tasks import *
from openai import OpenAI
from collections import defaultdict
import networkx as nx
import random, os
import pickle
import walker
from collections import Counter
from rdkit import Chem
from rdkit.Chem import rdFMCS
from collections import Counter
from ogb.lsc import PCQM4Mv2Dataset
import random
import pickle
import networkx as nx
from collections import defaultdict

response_dict = defaultdict(dict)
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(0)

In [ ]:

def smiles_to_networkx(smiles, perm=False):
    mol = Chem.MolFromSmiles(smiles)
    G = nx.Graph()
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), label=atom.GetSymbol())
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), label=bond.GetBondType())
    if perm:
        perm = list(range(G.number_of_nodes()))
        random.shuffle(perm)
        G = nx.relabel_nodes(G, dict(zip(range(G.number_of_nodes()), perm)))
    return G

molecular_set = PCQM4Mv2Dataset(root='dataset', only_smiles=True)
graphs = []
print(len(molecular_set))
node_num_dict = defaultdict(list)
knt = 0
for mol in molecular_set:
    knt += 1
    if knt%1000 == 0:
        print(knt)
    g = smiles_to_networkx(mol[0])
    node_num_dict[g.number_of_nodes()].append(g)

with open('source/molecular_graphs.pkl', 'wb') as f:
    pickle.dump(node_num_dict, f)

In [ ]:
triple = []
with open('dataset/rel_triples_1', 'r', encoding='UTF-8') as f:
    for line in f.readlines():
        head, rel, tail = line.strip().split('\t')
        head = head.split('resource/')[-1].replace('_', ' ')
        tail = tail.split('resource/')[-1].replace('_', ' ')
        rel = rel.split('ontology/')[-1]
        triple.append((head, tail, rel))
# summarize the enttiy and relation
entity = set()
relation = set()
for h, t, r in triple:
    entity.add(h)
    entity.add(t)
    relation.add(r)
len(entity), len(relation), len(triple)
# construct a network graph based on the triplet
import networkx as nx
G = nx.DiGraph()
for h, t, r in triple:
    G.add_edge(h, t, relation=r)
# construct a network graph based on the triplet
import networkx as nx
G = nx.DiGraph()
for h, t, r in triple:
    G.add_edge(h, t, relation=r)
G.number_of_edges(), G.number_of_nodes()